In [28]:
from ema_workbench import Scenario
from ema_workbench.analysis import prim
from problem_formulation import get_model_for_problem_formulation
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench import load_results
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [12]:
dike_model, planning_steps = get_model_for_problem_formulation("Gorssel")

# Scenario selection

In [13]:
ACTOR = "genscen_Gorssel_50000_08-06-2021-01-09-07.tar.gz"
experiments, outcomes = load_results("data/generated/" + ACTOR)
outcomes_df = pd.DataFrame(outcomes)

In [14]:
outcomes_df.columns.values.tolist()

['Difference in Expected Annual Damage Gorssel-Deventer',
 'Difference in Expected Number of Deaths Gorssel-Deventer',
 'Expected Annual Damage Gorssel']

In [18]:
DIFDAMAGE = outcomes_df.columns[0]
DIFDEATHS = outcomes_df.columns[1]
DAMAGE = outcomes_df.columns[2]

In [19]:
results = pd.concat([experiments, outcomes_df], axis=1)

In [24]:
damage_df = results.loc[results[DAMAGE] > np.percentile(a=outcomes[DAMAGE], q=90)]
difdamage_df = results.loc[results[DIFDAMAGE] > np.percentile(a=outcomes[DIFDAMAGE], q=90)]
difdeaths_df = results.loc[results[DIFDEATHS] > np.percentile(a=outcomes[DIFDEATHS], q=90)]
len(set(damage_df["scenario"].tolist()) & set(difdamage_df["scenario"].tolist()) & set(difdeaths_df["scenario"].tolist()))

0

In [26]:
selected = [damage_df.sample(n=1, random_state=1), difdamage_df.sample(n=1, random_state=1), difdeaths_df.sample(n=1, random_state=1)]

In [ ]:
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

# Optimisation

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,0,0,0], maximum=[3, 2,1.01,1.01]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(model.outcomes)
    
    results.append(optimize(scenario, 1e4, dike_model, convergence_metrics, epsilons))


# Regret

In [ ]:
thresholds = {}

# Satisficing

In [ ]:
thresholds = {}